***Copyright 2024 by the authors' university which is unnamed because of KDD's anonymity requirement.***

##**Connect to Drive**

In [ ]:
from google.colab import drive
from google.colab import files

drive.mount('/content/gdrive/')
print("-"*80)

!ls "/content/gdrive/My Drive/Colab Notebooks/Research/CFE"

import os
os.chdir("/content/gdrive/My Drive/Colab Notebooks/Research/CFE")
print("Current dir: ", os.getcwd())

##**Imports**

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings

from sklearn.svm import LinearSVC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, precision_recall_curve, roc_curve, auc
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

# Suppress warnings
warnings.filterwarnings("ignore")

##**Linear SVM**

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/CFE_datasets/Obesity Classification.csv')

df['Label'] = df['Label'].map({'Normal Weight': 0, 'Overweight': 0, 'Obese': 0, 'Underweight': 1})
df['Gender'] = df['Gender'].map({'Female': 0, 'Male': 1})
df = df.drop(columns=['ID'])
df = df.dropna()
df['Height'] = df['Height']/100

X = df.loc[:, df.columns != 'Label']
y = df['Label']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

params = {
    'C': [1, 5, 10, 25, 50, 100]
}

model = LinearSVC(random_state=42)
cv = StratifiedKFold(n_splits=10)
grid_search = GridSearchCV(estimator=model, param_grid=params, n_jobs=-1, cv=cv, scoring='f1_macro')

grid_search.fit(X_train, y_train)
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)
rf_best = grid_search.best_estimator_
print("Best Estimator:", rf_best)

Best Parameters: {'C': 1}
Best Score: 1.0
Best Estimator: LinearSVC(C=1, random_state=42)


In [ ]:
best = LinearSVC(C=5, random_state=42)
best.fit(X, y)

train_predictions = best.predict(X)
train_accuracy = accuracy_score(y, train_predictions)
print("Training Set Accuracy:", train_accuracy)

Training Set Accuracy: 0.9259259259259259


##**Extracting weights and intercept values**

In [ ]:
b = best.intercept_
w = best.coef_

w = w.flatten()

##**Setting the original misclassified *x* feature vector**

In [ ]:
og_x = df.drop(columns=['Label'])
predicted_label = best.predict(og_x)

results_df = pd.DataFrame({'BMI_Predicted': predicted_label})
index_class = results_df.index[results_df['BMI_Predicted'] == 0]

og_x_reset = og_x.reset_index(drop=True)
filtered_records = og_x_reset.loc[index_class]
print(len(filtered_records))

53


##**Z3 implementation**

In [ ]:
!pip install z3
!pip install z3-solver

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 23.2 MB/s eta 0:00:00
  Created wheel for z3: filename=z3-0.2.0-py3-none-any.whl size=26629 sha256=68825b29bf9f22f9dc4dab50da5cee4eee60b854b604538f15e6d9c4694ac08f
  Stored in directory: /root/.cache/pip/wheels/5a/b2/60/55b07a5084cad7ab411e395fb1440a2b1a19598bff535a3955
Successfully built z3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 8.7 MB/s eta 0:00:00


##**Optimizer  -- step 1**

In [ ]:
import csv
from z3 import *
import time


def abs(x):
    return If(x < 0, -x, x)

def step1(W, B, og_x_np):
    n = W.shape[0]

    # Create a Z3 optimization instance
    opt = Optimize()

    # Create Z3 real variable for X'
    X = [Real(f'x_{i}') for i in range(n)]

    #age
    original_value = og_x_np[0]
    if original_value is not None:
        opt.add(X[0] == original_value)

    #gender
    original_value1 = og_x_np[1]
    if original_value1 is not None:
        opt.add(X[1] == original_value1)

    #height
    original_value2 = og_x_np[2]
    if original_value2 is not None:
        opt.add(X[2] == original_value2)

    #weight
    constraint1 = And(X[3] >= 50, X[3] <= 60)
    opt.add(constraint1)

    bmi_constraint = If(And(X[3]/(X[2]**2) >= 0, X[3]/(X[2]**2) < 18.5), And(X[4] >= 0, X[4] < 18.5), X[4] >= 18.5)
    opt.add(bmi_constraint)

    # Calculate the dot product W * X
    dot_product = Sum([W[i] * X[i] for i in range(n)])

    inequality = dot_product + B > 0
    opt.add(inequality)

    # Calculate the L1 distance between og_x and X
    l1_distance = Sum([abs(og_x_np[i] - X[i]) for i in range(n)])

    objective = l1_distance
    opt.minimize(objective)

    # Check if there's a solution
    if opt.check() == sat:
        model = opt.model()

        original_f = [og_x_np[i] for i in range(n)]

        # Get the optimized feature vector x_prime
        x_prime_optimized = [model[X[i]].as_decimal(15) for i in range(n)]
        print(f"Optimized x': {x_prime_optimized}")

        # Calculate the optimized distance
        l1_distance_1 = model.eval(objective).as_decimal(15)
        print(f"Cost: {l1_distance_1}")

        # Calculate the optimized distance
        cost_optimized = model.eval(objective).as_decimal(15)
        print(f"Objective: {cost_optimized}")

        return original_f, x_prime_optimized, l1_distance_1, cost_optimized
    else:
        print("````````````FAILED TO FIND AN OPTIMAL SOLUTION.`````````````````````")

        original_f = [og_x_np[i] for i in range(n)]
        return [original_f, '', '', '']

W = w
B = b[0]
output_file = "/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/BMI_l1/Step1.csv"

for i in range(len(filtered_records)):
    row = filtered_records.iloc[i]
    og_x_np = row.to_numpy()

    start_time = time.time()

    original_f, x_prime_optimized, l1_distance_1, cost_optimized = step1(W, B, og_x_np)

    end_time = time.time()
    runtime = end_time - start_time

    with open(output_file, 'a', newline='') as file:
        writer = csv.writer(file)
        data_row = [original_f] + [x_prime_optimized] + [l1_distance_1] + [cost_optimized] + [runtime]
        writer.writerow(data_row)

In [ ]:
def remove_question_mark(cell_value):
    return cell_value.replace('?', '') if isinstance(cell_value, str) else cell_value

def remove_question_c(cell_value):
    return cell_value.replace('[', '') if isinstance(cell_value, str) else cell_value

def remove_question_cc(cell_value):
    return cell_value.replace(']', '') if isinstance(cell_value, str) else cell_value

def convert_to_float(lst_str):
    lst_str = str(lst_str)
    lst = lst_str.split(", ")
    return np.array([float(item.strip("'")) for item in lst])

In [ ]:
column_names = ['Original_x', 'Optimal_x', 'Cost', 'Objective', 'Runtime']
df1 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/BMI_l1/Step1.csv', names = column_names)

cleaned_df = df1.applymap(remove_question_mark)
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].map(remove_question_c)
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].map(remove_question_cc)

cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].apply(convert_to_float)
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].map(remove_question_c)
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].map(remove_question_cc)
cleaned_df

optimized = cleaned_df.dropna()
left = cleaned_df[cleaned_df.isna().any(axis=1)]

optimized.to_excel('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/BMI_l1/Step1_optimized.xlsx', index=False)
left.to_excel('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/BMI_l1/Step1_failed.xlsx', index=False)

for i in range(len(optimized['Optimal_x'])):
  x = optimized['Optimal_x'].iloc[i]
  predicted_label = best.predict(x.reshape(1, -1))
  print(predicted_label )

##**Optimizer -- step 2**


In [ ]:
def step2(W, B, row):
    n = W.shape[0]

    # Create a Z3 optimization instance
    opt = Optimize()

    # Create Z3 real variable for X'
    X = [Real(f'x_{i}') for i in range(n)]

    #age
    original_value = row[0]
    if original_value is not None:
        opt.add(X[0] == original_value)

    #gender
    original_value1 = row[1]
    if original_value1 is not None:
        opt.add(X[1] == original_value1)

    #height
    original_value2 = row[2]
    if original_value2 is not None:
        opt.add(X[2] == original_value2)

    #weight
    slack_weight = Real('slack_weight')
    weight_slack_constraint = (slack_weight>=0)
    constraint1 = And(X[3] > 0, X[3] >= 50 - slack_weight, X[3] <= 60 + slack_weight)
    opt.add(constraint1, weight_slack_constraint)

    bmi_constraint = If(And(X[3]/(X[2]**2) >= 0, X[3]/(X[2]**2) < 18.5), And(X[4] >= 0, X[4] < 18.5), X[4] >= 18.5)
    opt.add(bmi_constraint)

    dot_product = Sum([W[i] * X[i] for i in range(n)])

    inequality = dot_product + B > 0
    opt.add(inequality)

    objective = slack_weight
    opt.minimize(objective)

    # Check if there's a solution
    if opt.check() == sat:
        model = opt.model()

        original_f = [row[i] for i in range(n)]

        # Get the optimized feature vector x_prime
        x_prime_optimized = [model[X[i]].as_decimal(15) for i in range(n)]
        print(f"Optimized x': {x_prime_optimized}")

        # Calculate the optimized distance
        cost_optimized = model.eval(objective).as_decimal(15)
        print(f"Objective: {cost_optimized}")

        slack_weight = model.eval(slack_weight).as_decimal(15)
        print(f"Slack weight: {slack_weight}")

        return original_f, x_prime_optimized, cost_optimized, cost_optimized, slack_weight
    else:
        print("````````````FAILED TO FIND AN OPTIMAL SOLUTION.`````````````````````")

        original_f = [row[i] for i in range(n)]
        return [original_f, '', '', '', '']

W = w
B = b[0]

output_file1 = "/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/BMI_l1/Step2.csv"
input_file = pd.read_excel('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/BMI_l1/Step1_failed.xlsx')

input_file['Original_x'] = input_file['Original_x'].map(remove_question_c)
input_file['Original_x'] = input_file['Original_x'].map(remove_question_cc)

input_file['Original_x'] = input_file['Original_x'].apply(convert_to_float)

for i in range(len(input_file['Original_x'])):
    row = input_file['Original_x'].iloc[i]
    start_time = time.time()

    original_f, x_prime_optimized, cost, cost_optimized, slack_weight = step2(W, B, row)

    end_time = time.time()
    runtime = end_time - start_time

    with open(output_file1, 'a', newline='') as file:
        writer = csv.writer(file)
        data_row = [original_f] + [x_prime_optimized] + [cost] + [cost_optimized] + [runtime] + [slack_weight]
        writer.writerow(data_row)

Optimized x': ['38', '1', '1.9', '45.004088883283128?', '17.5']
Objective: 4.995911116716871?
Slack weight: 4.995911116716871?
Optimized x': ['37', '1', '1.9', '44.952261115628915?', '17.5']
Objective: 5.047738884371084?
Slack weight: 5.047738884371084?
Optimized x': ['36', '1', '1.9', '44.900433347974703?', '17.5']
Objective: 5.099566652025296?
Slack weight: 5.099566652025296?
Optimized x': ['35', '1', '1.9', '44.848605580320490?', '17.5']
Objective: 5.151394419679509?
Slack weight: 5.151394419679509?


In [ ]:
# Read the CSV file into a DataFrame
column_names = ['Original_x', 'Optimal_x', 'Cost', 'Objective', 'Runtime', 'Slack_weights']
df_step2 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/BMI_l1/Step2.csv', names = column_names)

def remove_question_mark(cell_value):
    return cell_value.replace('?', '') if isinstance(cell_value, str) else cell_value

def remove_question_c(cell_value):
    return cell_value.replace('[', '') if isinstance(cell_value, str) else cell_value

def remove_question_cc(cell_value):
    return cell_value.replace(']', '') if isinstance(cell_value, str) else cell_value

def convert_to_float(lst_str):
    lst_str = str(lst_str)
    lst = lst_str.split(", ")
    return np.array([float(item.strip("'")) for item in lst])

df_step2 = df_step2.applymap(remove_question_mark)
df_step2['Optimal_x'] = df_step2['Optimal_x'].map(remove_question_c)
df_step2['Optimal_x'] = df_step2['Optimal_x'].map(remove_question_cc)

# Apply the function to each row in the DataFrame
df_step2['Optimal_x'] = df_step2['Optimal_x'].apply(convert_to_float)
df_step2['Optimal_x'] = df_step2['Optimal_x'].map(remove_question_c)
df_step2['Optimal_x'] = df_step2['Optimal_x'].map(remove_question_cc)

df_step2.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/BMI_l1/Step2.csv', index=False)

for i in range(len(df_step2['Optimal_x'])):
  x = df_step2['Optimal_x'].iloc[i]
  predicted_label = best.predict(x.reshape(1, -1))
  print(predicted_label)

[1]
[1]
[1]
[1]


##**Optimizer -- Step 3**

In [ ]:
import csv
from z3 import *
import time

def step3(W, B, row, slack_value):
    n = W.shape[0]

    # Create a Z3 optimization instance
    opt = Optimize()

    # Create Z3 real variable for X'
    X = [Real(f'x_{i}') for i in range(n)]

    #age
    original_value = row[0]
    if original_value is not None:
        opt.add(X[0] == original_value)

    #gender
    original_value1 = row[1]
    if original_value1 is not None:
        opt.add(X[1] == original_value1)

    #height
    original_value2 = row[2]
    if original_value2 is not None:
        opt.add(X[2] == original_value2)

    #weight
    constraint1 = And(X[3] >= 50 - slack_value, X[3] <= 60 + slack_value)
    opt.add(constraint1)

    bmi_constraint = If(And(X[3]/(X[2]**2) >= 0, X[3]/(X[2]**2) < 18.5), And(X[4] >= 0, X[4] < 18.5), X[4] >= 18.5)
    opt.add(bmi_constraint)

    # Calculate the dot product W * X
    dot_product = Sum([W[i] * X[i] for i in range(n)])
    inequality = dot_product + B > 0
    opt.add(inequality)

    l0 = Sum([If(row[i] != X[i], 1, 0) for i in range(n)])

    objective = l0
    opt.minimize(objective)

    # Check if there's a solution
    if opt.check() == sat:
        model = opt.model()

        original_f = [row[i] for i in range(n)]

        x_prime_optimized = [model[X[i]].as_decimal(15) for i in range(n)]
        print(f"Optimized x': {x_prime_optimized}")

        l1_distance_1 = model.eval(objective)
        print(f"Distance: {l1_distance_1}")

        # Calculate the optimized distance
        cost_optimized = model.eval(objective)
        print(f"Objective: {cost_optimized}")

        return original_f, x_prime_optimized, l1_distance_1, cost_optimized
    else:
        print("````````````FAILED TO FIND AN OPTIMAL SOLUTION.`````````````````````")

        original_f = [row[i] for i in range(n)]
        return [original_f, '', '', '']

W = w
B = b[0]

out = '/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/BMI_l0/Step3.csv'
last = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/BMI_l1/Step2.csv')

last['Original_x'] = last['Original_x'].map(remove_question_c)
last['Original_x'] = last['Original_x'].map(remove_question_cc)
last['Original_x'] = last['Original_x'].apply(convert_to_float)

for i in range(len(last['Original_x'])):
    row = last['Original_x'].iloc[i]
    slack_value = last['Slack_weights'].iloc[i]
    start_time = time.time()

    original_f, x_prime_optimized, l1_distance_1, cost_optimized = step3(W, B, row, slack_value)

    end_time = time.time()
    runtime = end_time - start_time

    with open(out, 'a', newline='') as file:
        writer = csv.writer(file)
        data_row = [original_f] + [x_prime_optimized] + [l1_distance_1] + [cost_optimized] + [runtime]
        writer.writerow(data_row)

In [ ]:
# Read the CSV file into a DataFrame
column_names = ['Original_x', 'Optimal_x', 'Cost', 'Objective', 'Runtime']
df_step3 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/BMI_l0/Step3.csv', names = column_names)

df_step3 = df_step3.applymap(remove_question_mark)
df_step3['Optimal_x'] = df_step3['Optimal_x'].map(remove_question_c)
df_step3['Optimal_x'] = df_step3['Optimal_x'].map(remove_question_cc)

df_step3['Optimal_x'] = df_step3['Optimal_x'].apply(convert_to_float)
df_step3['Optimal_x'] = df_step3['Optimal_x'].map(remove_question_c)
df_step3['Optimal_x'] = df_step3['Optimal_x'].map(remove_question_cc)

df_step3.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/BMI_l0/Step3.csv', index=False)

for i in range(len(df_step3['Optimal_x'])):
  x = df_step3['Optimal_x'].iloc[i]
  predicted_label = best.predict(x.reshape(1, -1))
  print(predicted_label)

## **Average runtimes for l0 and l1**

In [ ]:
d3_prior = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/BMI_l1/Step2.csv')
d3_initial = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/BMI_l1/Step1_failed.csv')
d1 = pd.read_excel('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/BMI_l1/Step1_optimized.xlsx')
d = pd.concat([d1, d3_initial])

l1_runtime = d.loc[:, 'Runtime'].mean()
print('L1 runtime', l1_runtime)

In [ ]:
final_l0 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/BMI_l0/Step3.csv')
l0_runtime = final_l0.loc[:, 'Runtime'].mean()

print('L0 runtime', l0_runtime)

## **Percentage of relaxations**

In [ ]:
relaxed_df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/BMI_l1/Step2.csv')
needed_relaxing = len(relaxed_df)

subset_of_experiment = len(filtered_records)

percentage_of_relaxations = (needed_relaxing/subset_of_experiment)*100
print(percentage_of_relaxations)

7.547169811320755


### **Average percentage of relaxed soft constraints**

In [ ]:
relaxed_df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/BMI_l1/Step2.csv')
number_of_slacks_per_record = 1

count = 0
total_sum = 0
for i in range(len(relaxed_df['Slack_weights'])):
    if relaxed_df['Slack_weights'].iloc[i] != 0:
        count = 1
    total_sum += (count / number_of_slacks_per_record)

average_per_dataset = total_sum / len(relaxed_df) * 100

print(average_per_dataset)


100.0
